In [2]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from scipy.sparse import csr_matrix
import random as rnd
import pickle
import sys


from datetime import datetime


from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.ensemble import IsolationForest, RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, auc, confusion_matrix, f1_score, mean_squared_error, plot_confusion_matrix
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.multiclass import OneVsRestClassifier, OutputCodeClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import label_binarize
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

### CONSTANTS
SEED = 123
NUM_FEATURES = 10
TRAIN_PCT = 0.8

MAX_DEPTH = 4
MAX_ITER = 300
N_NEIGHBORS = 5



In [3]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [4]:
def generate_random_array(median,std,size):
    output=[0]*size    
    for index in range(0,size//2):
        random_nr=rnd.uniform(-std,std)
        output[2*index]=median+random_nr
        output[2*index+1]=median-random_nr
    if(size % 2 != 0):
        output[size-1]=rnd.uniform(median-std,median+std)
    return output



In [6]:
def selectFeatures(x, y, train_size_pct=0.75):
    """
    selectFeatures
        x : The features of the dataset to be used for predictions
        y : The target class for each row in "x"
        train_size_pct : (default = 0.75) In the tange (0.0, 1.0), the ratio by which to split the data for training and testing
        @return (list) The names of the selected features
    """

    # Create classifiers
    rf = RandomForestClassifier(max_depth=MAX_DEPTH, criterion='entropy', random_state=SEED)
    et = ExtraTreesClassifier(max_depth=MAX_DEPTH, criterion='entropy', random_state=SEED)
    dectree = DecisionTreeClassifier(max_depth=MAX_DEPTH, random_state=SEED)

    classifier_mapping = {
        "RandomForest" : rf,
        "ExtraTrees" : et,
        "DecisionTree" : dectree
    }

    ### Split the dataset
    X_train_fs, X_test_fs, Y_train_fs, Y_test_fs = train_test_split(x, y, train_size=train_size_pct)

    model_features = {}

    for model_name, model in classifier_mapping.items():
        print(f'[Training] {model_name}')
        start_train = datetime.now()
        model.fit(X_train_fs, Y_train_fs)
        print(">>> Training Time: {}".format(datetime.now() - start_train))
        model_features[model_name] = model.feature_importances_
        model_score = model.score(X_test_fs, Y_test_fs)
        print(f'>>> Training Accuracy : {model_score*100.0}')
        print("")

    cols = X_train_fs.columns.values
    feature_df = pd.DataFrame({'features': cols})
    for model_name, model in classifier_mapping.items():
        feature_df[model_name] = model_features[model_name]

    ### Grab the nlargest features (by score) from each ensemble group
    all_f = []
    for model_name, model in classifier_mapping.items():
        try:
            all_f.append(feature_df.nlargest(NUM_FEATURES, model_name))
        except KeyError as e:
            print(f'*** Failed to add features for {model_name} : {e}')

    result = []
    for i in range(len(all_f)):
        result.extend(all_f[i]['features'].to_list())		# Concat the top nlargest scores from all groups into one list

    # selected_features contains the ensemble results for best features
    selected_features = list(set(result))					# Drop duplicate fields from the list

    return selected_features


    

In [7]:
def evaluateIndividualClassifiers(x, y, train_size_pct):
    """
    evaluateIndividualClassifiers
        x : The features of the dataset to be used for predictions
        y : The target class for each row in "x"
        train_size_pct : {float in the range(0.0, 1.0)} the percentage of the dataset that should be used for training
    """

    max_depth_x2 = MAX_DEPTH * 2
    max_iter_x2 = MAX_ITER * 2
    n_neighbors_x2 = N_NEIGHBORS * 2
    n_neighbors_d2 = N_NEIGHBORS // 2

    rf = RandomForestClassifier(max_depth=MAX_DEPTH, criterion='entropy', random_state=SEED)
    rf_x2 = RandomForestClassifier(max_depth=max_depth_x2, criterion='entropy', random_state=SEED)
    et = ExtraTreesClassifier(max_depth=MAX_DEPTH, criterion='entropy', random_state=SEED)
    dectree = DecisionTreeClassifier(max_depth=MAX_DEPTH, random_state=SEED)
    knn = KNeighborsClassifier(n_neighbors=N_NEIGHBORS)
    knn_x2 = KNeighborsClassifier(n_neighbors=n_neighbors_x2)
    knn_d2 = KNeighborsClassifier(n_neighbors=n_neighbors_d2)
    mlpnn = MLPClassifier(max_iter=MAX_ITER)
    mlpnnE = MLPClassifier(max_iter=MAX_ITER, early_stopping=True)
    mlpnn_x2 = MLPClassifier(max_iter=max_iter_x2)
    mlpnnE_x2 = MLPClassifier(max_iter=max_iter_x2, early_stopping=True)

    classifier_mapping = {
        f'RandomForest-{MAX_DEPTH}' : rf,
        f'RandomForest-{max_depth_x2}' : rf_x2,
        f'ExtraTrees-{MAX_DEPTH}' : et,
        f'DecisionTree-{MAX_DEPTH}' : dectree,
        f'KNeighbors-{N_NEIGHBORS}' : knn,
        f'KNeighbors-{n_neighbors_x2}' : knn_x2,
        f'KNeighbors-{n_neighbors_d2}' : knn_d2,
        f'MLP-{MAX_ITER}' : mlpnn,
        f'MLP-{MAX_ITER}-early' : mlpnnE,
        f'MLP-{max_iter_x2}' : mlpnn_x2,
        f'MLP-{max_iter_x2}-early' : mlpnnE_x2,
    }

    for model_name, model in classifier_mapping.items():

        train_test_model(model_name, model, x, y, train_size_pct)


In [8]:
def train_test_model(model_name, model, x, y, train_size_pct):

    # Split the data
    X_train, X_test, Y_train, Y_test = train_test_split(x, y, train_size=train_size_pct)

    # Training
    print(f'\n[Training] {model_name}')
    start_train = datetime.now()
    model.fit(X_train, Y_train)
    print(f'>>> Training time: {datetime.now() - start_train}')

    ### Analyze Training
    train_acc = model.score(X_train, Y_train)
    print(f'>>> Training accuracy: {train_acc}')

    ### Testing
    start_predict = datetime.now()
    y_pred = model.predict(X_test)
    print(f'>>> Testing time: {datetime.now() - start_predict}')

    ### Analyze Testing
    calculateMetrics(Y_test, y_pred)



In [9]:
def calculateMetrics(y_test, y_pred):
    acc = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred, average="macro")
    precision = precision_score(y_test, y_pred, average="macro", zero_division=0)
    mse = mean_squared_error(y_test, y_pred)
    f1score = f1_score(y_pred, y_test, average='weighted')
    print(">>> Metrics")
    print(f'- Accuracy  : {acc}')
    print(f'- Recall    : {recall}')
    print(f'- Precision : {precision}')
    print(f'- MSE       : {mse}')
    print(f'- F1 Score  : {f1score}')

    return [round(acc, 6), round(recall, 6), round(precision, 6), round(mse, 6), round(f1score, 6)]

In [10]:
traffic = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/TrafficLabelling/combined.csv")
print(f'Number of Rows: {traffic.shape[0]}')
print(f'Number of Columns: {traffic.shape[1]}')
pd.set_option('display.max_colwidth', None)
traffic.head()

FileNotFoundError: ignored

In [ ]:
# Remove duplicate entries
traffic.drop_duplicates(subset=None, keep='first', inplace=True, ignore_index=False)

# reformat Timestamp to Unix time
# traffic[' Timestamp'] = pd.to_datetime(traffic[' Timestamp'])
# traffic[' Timestamp']= traffic[' Timestamp'].values.astype(np.int64) // 10 ** 9

# remove Timestamp column
traffic = traffic.drop(columns=[' Timestamp'])

# Remove constant columns
traffic = traffic.loc[:, traffic.apply(pd.Series.nunique) != 1]

# Remove column with NaN or Inf
traffic = traffic[~traffic.isin([np.nan, np.inf, -np.inf]).any(1)]

# remove Flow ID
traffic = traffic.drop(columns=['Flow ID'])

# reformat Label
traffic[' Label'] = traffic[' Label'].astype('category')
traffic[' Label'] = traffic[' Label'].cat.codes

# add new "is Attacker?" column
traffic[' is Attacker?'] = np.where(traffic[' Label'] == 0, 0, 1)


In [ ]:
columns = list(traffic.columns)
columns


[' Source IP',
 ' Source Port',
 ' Destination IP',
 ' Destination Port',
 ' Protocol',
 ' Flow Duration',
 ' Total Fwd Packets',
 ' Total Backward Packets',
 'Total Length of Fwd Packets',
 ' Total Length of Bwd Packets',
 ' Fwd Packet Length Max',
 ' Fwd Packet Length Min',
 ' Fwd Packet Length Mean',
 ' Fwd Packet Length Std',
 'Bwd Packet Length Max',
 ' Bwd Packet Length Min',
 ' Bwd Packet Length Mean',
 ' Bwd Packet Length Std',
 'Flow Bytes/s',
 ' Flow Packets/s',
 ' Flow IAT Mean',
 ' Flow IAT Std',
 ' Flow IAT Max',
 ' Flow IAT Min',
 'Fwd IAT Total',
 ' Fwd IAT Mean',
 ' Fwd IAT Std',
 ' Fwd IAT Max',
 ' Fwd IAT Min',
 'Bwd IAT Total',
 ' Bwd IAT Mean',
 ' Bwd IAT Std',
 ' Bwd IAT Max',
 ' Bwd IAT Min',
 'Fwd PSH Flags',
 ' Fwd URG Flags',
 ' Fwd Header Length',
 ' Bwd Header Length',
 'Fwd Packets/s',
 ' Bwd Packets/s',
 ' Min Packet Length',
 ' Max Packet Length',
 ' Packet Length Mean',
 ' Packet Length Std',
 ' Packet Length Variance',
 'FIN Flag Count',
 ' SYN Flag Coun

In [ ]:
# create ip address dict {ip_addr: node_id}
ips = []        # list of ip addresses.
for ip in traffic[' Source IP']:
    if ip not in ips:
        ips.append(ip)
        
for ip in traffic[' Destination IP']:
    if ip not in ips:
        ips.append(ip)
                
ips_dict = {ips[i]: i for i in range(len(ips))}   # dictionary of ip addresses { ip_addr: node_id }
print("Number of nodes: " + str(len(ips_dict)))


Number of nodes: 18598


In [ ]:
# adding a group_size column to the original dataframe
traffic['group_size'] = traffic.groupby([' Source IP'])[' Destination IP'].transform(len)
print(traffic)

              Source IP   Source Port  ...  is Attacker?  group_size
0        104.16.207.165           443  ...             0          29
1         104.16.28.216            80  ...             0         114
2         104.16.28.216            80  ...             0         114
3         104.17.241.25           443  ...             0           2
4        104.19.196.102           443  ...             0         120
...                 ...           ...  ...           ...         ...
2660372   192.168.10.14         51114  ...             0      129475
2660373   192.168.10.16         24054  ...             0      114034
2660374  23.208.163.130           443  ...             0          51
2660375   192.168.10.14         51694  ...             0      129475
2660376   192.168.10.14         57949  ...             0      129475

[2657447 rows x 77 columns]


In [ ]:
# define aggregation functions for all columns
aggregations = {col_name: "mean" for col_name in columns}
aggregations.pop(' Label', None)
aggregations.pop(' Source IP', None)
aggregations.pop(' Source Port', None)
aggregations.pop(' Destination IP', None)
aggregations.pop(' Destination Port', None)
aggregations.pop(' Protocol', None)
aggregations.pop(' is Attacker?', None)

aggregations.update({' is Attacker?': lambda x: 1 if 1 in set(x) else 0})
print(aggregations)


{' Flow Duration': 'mean', ' Total Fwd Packets': 'mean', ' Total Backward Packets': 'mean', 'Total Length of Fwd Packets': 'mean', ' Total Length of Bwd Packets': 'mean', ' Fwd Packet Length Max': 'mean', ' Fwd Packet Length Min': 'mean', ' Fwd Packet Length Mean': 'mean', ' Fwd Packet Length Std': 'mean', 'Bwd Packet Length Max': 'mean', ' Bwd Packet Length Min': 'mean', ' Bwd Packet Length Mean': 'mean', ' Bwd Packet Length Std': 'mean', 'Flow Bytes/s': 'mean', ' Flow Packets/s': 'mean', ' Flow IAT Mean': 'mean', ' Flow IAT Std': 'mean', ' Flow IAT Max': 'mean', ' Flow IAT Min': 'mean', 'Fwd IAT Total': 'mean', ' Fwd IAT Mean': 'mean', ' Fwd IAT Std': 'mean', ' Fwd IAT Max': 'mean', ' Fwd IAT Min': 'mean', 'Bwd IAT Total': 'mean', ' Bwd IAT Mean': 'mean', ' Bwd IAT Std': 'mean', ' Bwd IAT Max': 'mean', ' Bwd IAT Min': 'mean', 'Fwd PSH Flags': 'mean', ' Fwd URG Flags': 'mean', ' Fwd Header Length': 'mean', ' Bwd Header Length': 'mean', 'Fwd Packets/s': 'mean', ' Bwd Packets/s': 'mean'

In [ ]:
# low activity grouping
low_activities = traffic[traffic['group_size'] < 10]
low_activities = low_activities.groupby(' Source IP', as_index=False).agg(aggregations)
low_activities



,Source IP,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,is Attacker?
0,0.0.0.0,3.668571e+07,2.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.054517,3.670000e+07,0.000000,3.670000e+07,3.670000e+07,3.670000e+07,3.670000e+07,0.000000,3.670000e+07,3.670000e+07,0.0,0.0,0.00000,0.0,0.0,0.0,0,0.000000,0.000000,0.054517,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0.0,0.0,0.00,0,0,0.0,0.000000,0.0,0.0,0.000000,2.000000,0.0,0.000000,0.0,-1.00,-1.00,0.000000,0.0,0.0,0.0,0.0,0.0,36700000.0,0.0,36700000.0,36700000.0,0
1,1.1.70.73,4.511675e+04,2.000000,0.250000,3.0,0.0,1.5,1.5,1.5,0.000000,0.0,0.0,0.0,0.0,8.720930e+03,4244.011538,2.269288e+04,31693.763375,4.510375e+04,2.820000e+02,4.511675e+04,4.511675e+04,0.000000,4.511675e+04,4.511675e+04,0.0,0.0,0.00000,0.0,0.0,0.0,0,58.000000,8.000000,4242.617935,1.393604,1.5,1.5,1.5,0.000000,0.0,0.0,0.0,0,0.0,1.0,0.25,0,0,0.0,2.250000,1.5,0.0,58.000000,2.000000,3.0,0.250000,0.0,4454.25,59.25,0.250000,29.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1.193.219.21,6.480000e+02,3.000000,0.000000,258.0,0.0,258.0,0.0,86.0,148.956369,0.0,0.0,0.0,0.0,3.981481e+05,4629.629630,3.240000e+02,452.548340,6.440000e+02,4.000000e+00,6.480000e+02,3.240000e+02,452.548340,6.440000e+02,4.000000e+00,0.0,0.0,0.00000,0.0,0.0,0.0,0,96.000000,0.000000,4629.629630,0.000000,0.0,258.0,64.5,129.000000,16641.0,0.0,0.0,0,0.0,1.0,0.00,0,0,0.0,86.000000,86.0,0.0,96.000000,3.000000,258.0,0.000000,0.0,61.00,-1.00,1.000000,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1.193.219.24,1.661294e+05,2.000000,0.800000,258.0,0.0,258.0,0.0,129.0,182.433549,0.0,0.0,0.0,0.0,4.301911e+07,333556.025118,4.166945e+04,83140.944944,1.660928e+05,1.680000e+01,1.661180e+05,1.661180e+05,0.000000,1.661180e+05,1.661180e+05,165574.8,82787.4,117078.21534,165574.2,0.6,0.8,0,64.000000,16.000000,333481.472436,74.552682,0.0,258.0,137.6,140.230707,19969.2,0.0,0.8,0,0.0,1.0,0.00,0,0,0.2,199.520000,129.0,0.0,64.000000,2.000000,258.0,0.800000,0.0,65.00,-0.60,0.200000,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,1.226.51.14,4.217540e+04,1.400000,0.600000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,15641.023359,4.217540e+04,0.000000,4.217540e+04,4.217540e+04,4.212860e+04,4.212860e+04,0.000000,4.212860e+04,4.212860e+04,0.0,0.0,0.00000,0.0,0.0,0.0,0,44.800000,19.200000,7824.395543,7816.627816,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0.0,1.0,0.60,0,0,0.6,0.000000,0.0,0.0,44.800000,1.400000,0.0,0.600000,0.0,155.20,866.00,0.000000,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11180,98.158.54.105,2.570000e+02,3.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,11673.151750,1.285000e+

In [ ]:
# attackers grouping
atk_activities = traffic[traffic[' is Attacker?'] == 1]
atk_activities = atk_activities.groupby(' Source IP', as_index=False).agg(aggregations)
atk_activities


,Source IP,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,is Attacker?
0,172.16.0.1,2.956412e+07,4.017171,3.225490,158.521657,5247.863025,114.118665,2.095020,24.556664,45.803378,2830.399745,1.865424,905.920939,1211.513215,317182.121404,96192.541960,2.907503e+06,8.457482e+06,2.862411e+07,225024.546124,2.933547e+07,5.546394e+06,1.159011e+07,2.853765e+07,545075.753076,8.864288e+06,1.918876e+06,3.528664e+06,8.584444e+06,349821.684346,0.012117,0,117.212187,94.934009,86743.368924,9449.173036,0.987842,2834.562531,431.105388,902.858431,2.013992e+06,0.106744,0.012117,0,0.459680,0.434348,0.007167,0,0,0.614946,472.879368,24.556664,905.920939,117.212187,4.017171,158.521657,3.225490,5247.863025,7331.240992,200.162452,1.512595,27.414352,103275.711488,19432.330451,116967.629362,89209.731133,2.719980e+07,1.509965e+06,2.835524e+07,2.612062e+07,1
1,192.168.10.12,3.013027e+07,5.000000,5.000000,161.000000,128.000000,161.000000,0.000000,17.888889,53.666667,128.000000,0.000000,14.222222,42.666667,4.800454,17.423337,1.799611e+06,2.450991e+06,5.128946e+06,28969.000000,2.560000e+07,3.198221e+06,2.634245e+06,5.100000e+06,117.000000,3.010000e+07,3.759477e+06,2.322569e+06,5.150000e+06,318.500000,0.000000,0,164.000000,164.000000,8.711669,8.711669,0.000000,161.000000,15.210526,45.891876,4.212129e+03,0.000000,0.000000,0,0.500000,0.500000,0.500000,0,0,1.000000,16.055556,17.888889,14.222222,164.000000,5.000000,161.000000,5.000000,128.000000,14718.500000,110.000000,0.500000,32.000000,31839.100000,11126.267980,51587.500000,25455.500000,5.100000e+06,1.747064e+04,5.100000e+06,5.050000e+06,1
2,192.168.10.14,4.012817e+05,4.755981,5.057416,4431.124402,107.736842,700.636364,0.000000,204.045950,198.270746,87.186603,2.009569,29.362901,44.737745,47960.846025,93.667237,7.676260e+04,1.150069e+05,2.302638e+05,177.090909,4.011073e+05,1.874357e+05,3.316208e+04,2.314064e+05,168935.693780,4.001571e+05,1.950188e+05,3.872412e+04,2.304383e+05,169237.483254,0.000000,0,113.818182,109.129187,46.028187,47.639050,0.000000,702.645933,89.350356,165.367536,2.719614e+05,0.000000,0.000000,0,1.000000,0.000000,0.000000,0,0,0.382775,94.335594,204.045950,29.362901,113.818182,4.755981,4431.124402,5.057416,107.736842,8192.000000,207.535885,3.086124,22.679426,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1
3,192.168.10.15,4.511831e+05,4.245902,4.325137,3593.650273,95.816940,523.346995,0.098361,154.350291,170.575954,79.360656,2.393443,27.850208,41.478106,175412.000815,23297.527461,8.735663e+04,1.287442e+05,2.533632e+05,198.658470,4.509802e+05,2.140238e+05,3.211725e+04,2.546007e+05,195742.969945,4.499535e+05,2.215904e+05,3.766137e+04,2.535668e+05,196000.587432,0.000000,0,104.371585,93.715847,11649.187520,11648.339941,0.098361,525.642077,69.494889,134.913609,2.915180e+05,0.000000,0.000000,0,0.983607,0.016393,0.000000,0,0,0.434426,74.007528,154.350291,27.850208,104.371585,4.245902,359

In [ ]:
basic_flows = traffic.groupby([' Source IP', ' Source Port', ' Destination IP', ' Destination Port', ' Protocol'], as_index=False)

basic_flows = basic_flows.agg(aggregations)
basic_flows

,Source IP,Source Port,Destination IP,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,is Attacker?
0,0.0.0.0,0,224.0.0.22,0,0,36685713.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.054517,36700000.0,0.0000,36700000.0,36700000.0,36700000.0,36700000.0,0.0,36700000.0,36700000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054517,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,36700000.0,0.0,36700000.0,36700000.0,0
1,1.1.70.73,80,192.168.10.15,52772,6,344.0,2.0,0.0,12.0,0.0,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,34883.720930,5813.953488,344.0,0.0000,344.0,344.0,344.0,344.0,0.0,344.0,344.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40.0,0.0,5813.953488,0.000000,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.0,6.0,0.0,40.0,2.0,12.0,0.0,0.0,4380.0,-1.0,1.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1.1.70.73,80,192.168.10.17,48301,6,315.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,6349.206349,315.0,0.0000,315.0,315.0,315.0,315.0,0.0,315.0,315.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,6349.206349,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,2.0,0.0,0.0,0.0,4380.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1.1.70.73,80,192.168.10.19,35264,6,179391.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,16.723247,89695.5,126775.0535,179339.0,52.0,179391.0,179391.0,0.0,179391.0,179391.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,32.0,11.148831,5.574416,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,2.0,0.0,1.0,0.0,4677.0,240.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,1.1.70.73,80,192.168.10.19,35275,6,417.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,4796.163070,417.0,0.0000,417.0,417.0,417.0,417.0,0.0,417.0,417.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,0.0,4796.163070,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64.0,2.0,0.0,0.0,0.0,4380.0,-1.0,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1382955,99.192.248.32,80,192.168.10.5,49372,6,55.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,218181.818182,36363.636364,55.0,0.0000,55.0,55.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,20.0,18181.818182,18181.818182,6.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,9.0,6.0,6.0,20.0,1.0,6.0,1.0,6.0,46.0,256.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1382956,99.192.248.32,80,192.168.10.5,49419,6,45.0,1.0,1.0,6.0,6.0,6.0,6.0,6.0,0.0,6.0,6.0,6.0,0.0,266666.666667,44444.444444,45.0,0.0000,45.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20

In [ ]:
activity_flows = basic_flows.groupby([' Source IP'], as_index=False)
activity_flows = activity_flows.agg(aggregations)

# reformat Source IP
activity_flows = activity_flows.replace({" Source IP": ips_dict})
print(activity_flows)

atkers = activity_flows.groupby([' is Attacker?']).get_group(1)
non_atkers = activity_flows.groupby([' is Attacker?']).get_group(0)
print("Number of attackers: " + str(len(atkers)))
print("Number of non_atkers " + str(len(non_atkers)))



        Source IP   Flow Duration  ...    Idle Min   is Attacker?
0           13147    3.668571e+07  ...  36700000.0              0
1             696    4.511675e+04  ...         0.0              0
2           15131    6.480000e+02  ...         0.0              0
3            1817    1.038322e+05  ...         0.0              0
4           13681    4.217540e+04  ...         0.0              0
...           ...             ...  ...         ...            ...
16508       13431    4.627100e+04  ...         0.0              0
16509        9872    4.732533e+04  ...         0.0              0
16510        2737    1.156000e+03  ...         0.0              0
16511       10465    8.496417e+03  ...         0.0              0
16512       10578    3.000000e+00  ...         0.0              0

[16513 rows x 71 columns]
Number of attackers: 10
Number of non_atkers 16503


In [ ]:
# create low profiles dataframe
num_ips = len(ips_dict)
num_src_ips = len(activity_flows[' Source IP'])
num_noflow_ips = num_ips - num_src_ips


no_flow_prof_columns = columns[5:-3]
print(no_flow_prof_columns)
num_low_ips = round(0.98*num_noflow_ips)   #### adjust benign/attackers ratio here
low_activities_prof = np.zeros((num_low_ips, len(no_flow_prof_columns)))
for col, attr in enumerate(no_flow_prof_columns):
  median = low_activities[attr].median()
  std = np.std(low_activities[attr])
  low_activities_prof[:, col] = generate_random_array(median, std, num_low_ips)


low_profile_df = pd.DataFrame(low_activities_prof, columns = no_flow_prof_columns)
low_profile_df[' is Attacker?'] = [0 for i in range(num_low_ips)]
low_profile_df[' Source IP'] = [num_src_ips + i for i in range(num_low_ips)]
low_profile_df



[' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets', 'Total Length of Fwd Packets', ' Total Length of Bwd Packets', ' Fwd Packet Length Max', ' Fwd Packet Length Min', ' Fwd Packet Length Mean', ' Fwd Packet Length Std', 'Bwd Packet Length Max', ' Bwd Packet Length Min', ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Fwd URG Flags', ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count', ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count', ' ECE Flag Count', ' Down/Up R

,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,is Attacker?,Source IP
0,1.047934e+06,0.811198,0.812650,1257.732190,-7.056445,25.470382,-1.410163,71.427614,-57.417110,15.868582,17.088212,-7.611809,-0.624601,-4.357612e+06,172554.807577,4578.762898,-610657.561377,9.760254e+05,-159422.567213,1.251354e+06,-437694.401235,425387.195188,-8.714976e+05,245353.207220,-87730.494981,-5094.169730,-33459.600727,7505.672936,-83325.958679,0.153809,0.0,58.979283,20.623185,170619.267238,-11740.012343,3.537292,-30.366054,19.622352,-34.586457,-5205.164619,0.032888,-0.294953,0.0,-0.004518,0.961313,0.641360,0.0,0.0,0.618562,76.069043,83.459744,-5.606146,36.328680,0.616589,88.415473,0.807195,-5.203535,-514.860883,5576.247938,0.600494,22.769483,-10909.657533,2228.925655,60244.540085,-28024.279072,1.085312e+06,43671.917274,-113000.204283,0,16513
1,-1.047780e+06,2.188802,0.687350,-1239.732190,10.056445,-13.470382,7.410163,-59.427614,57.417110,-12.868582,-14.088212,10.611809,0.624601,4.957612e+06,-18708.653737,-4444.762898,610657.561377,-9.758774e+05,159513.900547,-1.251346e+06,437701.401235,-425387.195188,8.715056e+05,-245348.064363,87730.494981,5094.169730,33459.600727,-7505.672936,83325.958679,-0.153809,0.0,17.020717,16.376815,-87141.006368,31457.274248,2.462708,42.366054,-7.622352,34.586457,5205.164619,-0.032888,0.294953,0.0,0.004518,1.038687,0.358640,0.0,0.0,0.381438,-58.069043,-71.459744,8.606146,39.671320,2.383411,-70.415473,0.692805,8.203535,764.860883,-5149.914605,-0.267161,17.230517,10909.657533,-2228.925655,-60244.540085,28024.279072,-1.085312e+06,-43671.917274,113000.204283,0,16514
2,7.455062e+05,2.098958,0.937870,830.096544,8.362910,87.083975,14.512443,45.311947,-43.206415,2.139036,3.605555,17.831160,0.015433,-2.719519e+06,172804.398712,-288516.072842,-177665.804494,-4.412109e+05,-307333.333367,-9.323553e+05,-439450.253629,470964.922539,-1.123753e+06,-106847.881680,13517.015568,35565.227255,14243.921040,-150454.302131,83279.747126,0.234037,0.0,47.359604,19.465540,-85642.219232,7146.704924,7.611590,-75.313887,40.486392,38.685601,1342.814112,-0.028464,0.207758,0.0,0.013354,1.047014,0.334821,0.0,0.0,0.686078,62.056559,10.037213,2.902945,63.911927,1.400432,674.085759,1.354667,14.789364,5161.157988,-10401.232926,0.062588,19.190766,14702.749946,5766.975036,-24478.858914,34884.430071,-5.841360e+04,-11118.689205,729924.675353,0,16515
3,-7.453522e+05,0.901042,0.562130,-812.096544,-5.362910,-75.083975,-8.512443,-33.311947,43.206415,0.860964,-0.605555,-14.831160,-0.015433,3.319519e+06,-18958.244872,288650.072842,177665.804494,4.413589e+05,307424.666700,9.323633e+05,439457.253629,-470964.922539,1.123761e+06,106853.024537,-13517.015568,-35565.227255,-14243.921040,150454.302131,-83279.747126,-0.234037,0.0,28.640396,17.534460,169120.480102,12570.556981,-1.611590,87.313887,-28.486392,-38.685601,-1342.814112,0.028464,-0.207758,0.0,-0.013354,0.952986,0.665179,0.0,0.0,0.313922,-44.056559,1.962

In [ ]:
# create attacker profiles dataframe

num_atk_ips = num_noflow_ips-num_low_ips

atker_prof = np.zeros((num_atk_ips, len(no_flow_prof_columns)))
for col, attr in enumerate(no_flow_prof_columns):
  median = atk_activities[attr].median()
  std = np.std(atk_activities[attr])
  atker_prof[:, col] = generate_random_array(median, std, num_atk_ips)


atker_profile_df = pd.DataFrame(atker_prof, columns = no_flow_prof_columns)
atker_profile_df[' is Attacker?'] = [1 for i in range(num_atk_ips)]
atker_profile_df[' Source IP'] = [num_src_ips+num_low_ips + i for i in range(num_atk_ips)]
atker_profile_df


,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,is Attacker?,Source IP
0,8.432509e+06,-6.398289,-17.189685,-9831.822621,-81.278364,-16.019222,1.838416,9.130216,74.942009,827.204914,1.836736,91.901567,-215.939747,159563.299005,-26656.713013,-2.489253e+05,1.899078e+06,-2.326964e+06,33256.343404,-6.983735e+06,1.049380e+06,-8.376605e+05,4.396967e+06,140317.372531,-8.199910e+06,4.141318e+05,5.473937e+05,3.010813e+06,64748.766608,0.019387,0.0,-350.812162,105.600314,30176.604882,-2623.181185,1.853658,231.906300,23.161956,198.504444,239783.608382,0.005297,-0.004626,0.0,0.793671,0.427644,0.365756,0.0,0.0,-0.063240,-1.751432,123.254812,140.280053,636.933052,1.220045,-3835.942086,-6.021032,333.547473,9911.703421,257.314349,-4.593144,24.614253,-112679.866037,55082.672756,-80146.448542,-64186.223486,-1.091989e+06,-262231.182322,1.344019e+06,1,18556
1,-2.533988e+06,15.925219,27.580434,13631.972894,300.647950,645.861242,-1.607312,139.621119,123.767464,-659.683116,2.519203,-44.740677,298.182226,-41951.487962,35185.678392,6.701428e+05,-8.576215e+05,7.266263e+06,-32849.623481,7.891642e+06,-6.203625e+05,9.085242e+05,-3.882375e+06,184969.710629,1.408854e+07,3.161140e+05,4.062215e+05,1.700459e+06,261276.734345,-0.019387,0.0,582.919642,117.986973,-25747.204539,6722.746220,-1.622554,720.400272,111.641756,49.409920,178042.842565,-0.005297,0.004626,0.0,0.563007,0.150027,-0.355332,0.0,0.0,1.274278,144.913826,25.496523,-93.119163,-404.825572,8.306885,7636.092359,16.411782,-114.177887,6166.622997,134.568773,8.586613,21.574348,127698.199370,-55082.672756,95164.781875,79204.556819,4.213261e+06,262231.182322,2.730309e+06,1,18557
2,-1.048650e+06,30.122722,-4.118905,14988.159331,181.484365,168.587529,-2.127155,121.383957,190.084289,-686.928484,0.469005,82.716522,-85.495226,119305.970929,-390.425609,-3.613808e+05,-1.256147e+06,3.522618e+06,-53303.874319,8.330704e+06,2.037544e+06,-2.636649e+06,-4.998087e+06,287147.425490,6.567639e+06,-1.286882e+05,6.649555e+05,2.100004e+06,95107.668112,0.001876,0.0,-270.541374,204.782629,-12096.232910,-84.929011,2.083972,-236.531499,-19.707053,187.109288,-236698.999733,0.008530,0.009829,0.0,0.877070,0.499166,0.118228,0.0,0.0,-0.661925,36.904776,76.230354,269.023171,82.784428,33.172198,6456.020249,-1.465952,706.473031,4871.222317,564.383449,-21.521701,19.938708,112296.350991,-56490.572791,-99302.841267,73572.208991,4.046744e+06,-799.084456,5.746373e+06,1,18558
3,6.947170e+06,-20.595792,14.509654,-11188.009058,37.885221,461.254490,2.358259,27.367378,8.625185,854.450283,3.886933,-35.555632,167.737705,-1694.159886,8919.390988,7.825982e+05,2.297603e+06,1.416681e+06,53710.594241,-7.422796e+06,-1.608526e+06,2.707512e+06,5.512679e+06,38139.657670,-6.790091e+05,8.589340e+05,2.886598e+05,2.611269e+06,230917.832841,-0.001876,0.0,502.648854,18.804658,16525.633253,4184.494046,-1.852868,1188.838071,154.510766,60.805077,654525.450679,-0.008530,-0.009829,0.0,0.479608,0.078504,-

In [ ]:
# combines all the dataframes
activity_flows_all = activity_flows.append(low_profile_df, ignore_index=True)
activity_flows_all = activity_flows_all.append(atker_profile_df, ignore_index=True)



In [ ]:

atkers = activity_flows_all.groupby([' is Attacker?']).get_group(1)
non_atkers = activity_flows_all.groupby([' is Attacker?']).get_group(0)
print("Number of attackers: " + str(len(atkers)))
print("Number of non_atkers " + str(len(non_atkers)))


Number of attackers: 52
Number of non_atkers 18546


In [ ]:
activity_flows_all = activity_flows_all.sort_values(by=' Source IP', ignore_index=True)
activity_flows_all

,Source IP,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,Bwd Packet Length Max,Bwd Packet Length Min,Bwd Packet Length Mean,Bwd Packet Length Std,Flow Bytes/s,Flow Packets/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Total,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Total,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Fwd URG Flags,Fwd Header Length,Bwd Header Length,Fwd Packets/s,Bwd Packets/s,Min Packet Length,Max Packet Length,Packet Length Mean,Packet Length Std,Packet Length Variance,FIN Flag Count,SYN Flag Count,RST Flag Count,PSH Flag Count,ACK Flag Count,URG Flag Count,CWE Flag Count,ECE Flag Count,Down/Up Ratio,Average Packet Size,Avg Fwd Segment Size,Avg Bwd Segment Size,Fwd Header Length.1,Subflow Fwd Packets,Subflow Fwd Bytes,Subflow Bwd Packets,Subflow Bwd Bytes,Init_Win_bytes_forward,Init_Win_bytes_backward,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,is Attacker?
0,0,2.958276e+02,1.655172,0.551724,9.241379,2.275862,6.344828,4.137931,4.873563,1.274152,2.275862,2.275862,2.275862,0.000000,7.443734e+05,173200.060082,254.298851,3.268862e+01,2.907586e+02,233.827586,2.620000e+02,2.324828e+02,3.940292e+01,2.603448e+02,204.620690,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,33.103448,11.034483,154810.571641,18389.488437,4.137931,6.344828,4.579310,0.986954,14.124138,0.000000,0.000000,0.0,0.000000,1.000000,0.482759,0.0,0.0,0.413793,6.620690,4.873563,2.275862,33.103448,1.655172,9.241379,0.551724,2.275862,31.586207,174.034483,0.482759,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.0,0
1,1,2.968569e+04,1.122807,0.877193,5.842105,4.789474,5.315789,5.315789,5.315789,0.000000,4.789474,4.789474,4.789474,0.000000,3.480764e+05,60856.512853,29685.692982,0.000000e+00,2.968569e+04,29685.692982,2.962772e+04,2.962772e+04,0.000000e+00,2.962772e+04,29627.719298,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,22.456140,17.543860,35588.410486,25268.102366,5.315789,5.315789,5.315789,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.877193,0.0,0.0,0.877193,7.973684,5.315789,4.789474,22.456140,1.122807,5.842105,0.877193,4.789474,29.008772,2768.675439,0.087719,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.0,0
2,2,1.920000e+02,1.500000,0.500000,9.000000,3.000000,6.000000,6.000000,6.000000,0.000000,3.000000,3.000000,3.000000,0.000000,1.936134e+05,32268.907562,192.000000,0.000000e+00,1.920000e+02,192.000000,1.750000e+02,1.750000e+02,0.000000e+00,1.750000e+02,175.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.0,30.000000,10.000000,17563.025212,14705.882355,6.000000,6.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.500000,0.0,0.0,0.500000,9.000000,6.000000,3.000000,30.000000,1.500000,9.000000,0.500000,3.000000,31.000000,164.000000,0.500000,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.0,0
3,3,1.322275e+03,2.050847,0.279661,26.500000,1.449153,19.483051,4.576271,9.133333,7.515555,1.423729,1.423729,1.423729,0.000000,1.011438e+06,204326.933754,579.338983,5.730325e+02,1.303220e+03,255.038136,1.307114e+03,7.785996e+02,5.956608e+02,1.296072e+03,380.025424,1.271186e-02,1.271186e-02,0.000000e+00,1.271186e-02,0.012712,0.025424,0.0,41.016949,5.593220,201936.127350,2390.806404,4.576271,19.483051,8.742857,7.009958,605.918402,0.000000,0.025424,0.0,0.000000,1.000000,0.152542,0.0,0.0,0.152542,11.812853,9.133333,1.423729,41.016949,2.050847,26.500000,0.279661,1.449153,31.533898,1701.580508,0.800847,20.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.0,0
4,4,1.114520e+03,1.510000,0.650000,64.720000,3.660000,39.880000,5.640000,20.296667,

In [ ]:
# (1) SELECT | ALL features
X = activity_flows_all.iloc[:, 1:(activity_flows_all.shape[1]-1)]
Y = activity_flows_all.iloc[:, -1]


In [ ]:
# create hypergraph
connection = traffic.groupby([' Source IP', ' Destination IP'], as_index=False)
connection = connection.agg({' Flow Duration' : 'count'})
src_ip_ids = [ips_dict[ip] for ip in connection[' Source IP']]
dst_ip_ids = [ips_dict[ip] for ip in connection[' Destination IP']]
edges = list(zip(src_ip_ids, dst_ip_ids))
hypergraph = {i: edges[i] for i in range(len(edges))}

In [ ]:
# # generate features
features = X
features = np.nan_to_num(features)
features = preprocessing.normalize(features)
features = csr_matrix(features)

# generate features from feature selection set
# X = X.fillna(0)
# selected_features = selectFeatures(X, Y)
# print(f'Selected Features "from All": {selected_features}')
# features = X[selected_features]
# print("Shape of features: ", features.shape)

# features = preprocessing.normalize(features)
# features = csr_matrix(features)


In [ ]:
# generate labels
labels = Y

In [ ]:
# train/test split

splits = {}
attackers_ids = list(atkers.index)
V = len(ips)
node_sorted = list(range(V))
rnd.shuffle(node_sorted)
train_size = round(0.8*V)
test_size = V - train_size
train_set, test_set = train_test_split(node_sorted, train_size=train_size, test_size=test_size)



splits['train'] = train_set
splits['test'] = test_set


In [ ]:
train_atk_count = 0
for idx, train_id in enumerate(train_set):
    if labels[train_id] == 1:
      train_atk_count += 1

test_atk_count = 0
for idx, test_id in enumerate(test_set):
    if labels[test_id] == 1:
      test_atk_count += 1

print("Number of atker in train set: ", train_atk_count)
print("Number of atker in test set: ", test_atk_count)

Number of atker in train set:  39
Number of atker in test set:  13


In [ ]:
# generate files
with open("hypergraph.pickle", "wb") as hypergraph_out:
    pickle.dump(hypergraph, hypergraph_out)
    hypergraph_out.close()
with open("features.pickle", "wb") as features_out:
    pickle.dump(features, features_out)
    features_out.close()
with open("labels.pickle", "wb") as labels_out:
    pickle.dump(labels, labels_out)
    labels_out.close()
with open("1.pickle", "wb") as splits_out:
    pickle.dump(splits, splits_out)
    splits_out.close()